[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stfc/janus-core/blob/main/docs/source/tutorials/cli/geomopt.ipynb)

# Geometry Optimization

## Set up environment (optional)

These steps are required for Google Colab, but may work on other systems too:

In [22]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

# ! pip uninstall torch torchaudio torchvision numpy -y
# ! uv pip install janus-core[all] data-tutorials torch==2.5.1 --system
# get_ipython().kernel.do_shutdown(restart=True)

## Command-line help and options

As with `janus singlepoint`, we can check the options for geometry optimisation:

In [1]:
! janus geomopt --help

                                                                                
 Usage: janus geomopt [OPTIONS]                                                 
                                                                                
 Perform geometry optimization and save optimized structure to file.            
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ *  --struct                               PATH              Path of          │
│                                                             structure to     │
│                                                             simulate.        │
│                                                             [default: None]  │
│                                                             [required]       │
│    --optimizer                            TEXT              Name of ASE      │
│                           

## Running geometry optimisation calculation

First, we'll build a structure to optimise, as we did for single point calculations, but add in a deformation:

In [2]:
from pathlib import Path

from ase.build import bulk
from ase.io import write
from weas_widget import WeasWidget

Path("data").mkdir(exist_ok=True)

NaCl = bulk("NaCl", "rocksalt", a=5.63, cubic=True)
NaCl[0].position = [1.5, 1.5, 1.5]

write("../data/NaCl-deformed.xyz", NaCl)

v=WeasWidget()
v.from_ase(NaCl)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

WeasWidget(children=(BaseWidget(atoms={'species': {'Na': 'Na', 'Cl': 'Cl'}, 'cell': [5.63, 0.0, 0.0, 0.0, 5.63…

Now we can optimise the geometry of this structure in a similar manner to running `janus singlepoint`:

In [3]:
! janus geomopt --struct ../data/NaCl-deformed.xyz --no-tracker

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

We can also change the optimisation function used, and specify an even lower force convergence criteria, `fmax` (the maximum force on all individual atoms):

<div class="alert alert-block alert-info">
<b>Tip:</b> The optimizer must be a class defined in ASE.
</div>


In [4]:
! janus geomopt --struct ../data/NaCl-deformed.xyz --optimizer FIRE --fmax 0.005 --no-tracker

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

As with single point calculations, this saves a results file, corresponding to the optimised structure, as well as a summary and log:

In [5]:
! ls janus_results/NaCl-deformed*

janus_results/NaCl-deformed-geomopt-log.yml
janus_results/NaCl-deformed-geomopt-summary.yml
janus_results/NaCl-deformed-opt.extxyz


We can now see the optimised structure:

In [6]:
from ase.io import read
from weas_widget import WeasWidget

traj = read("janus_results/NaCl-deformed-opt.extxyz")

v=WeasWidget()
v.from_ase(traj)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

WeasWidget(children=(BaseWidget(atoms={'species': {'Na': 'Na', 'Cl': 'Cl'}, 'cell': [5.63, 0.0, 0.0, 0.0, 5.63…

## Trajectories

We can also save the trajectory during optimisation:

In [7]:
! janus geomopt --struct ../data/NaCl-deformed.xyz --write-traj --no-tracker

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

This creates an additional file, `janus_results/NaCl-deformed-traj.extxyz`:

In [8]:
! ls janus_results/NaCl-deformed*

janus_results/NaCl-deformed-geomopt-log.yml
janus_results/NaCl-deformed-geomopt-summary.yml
janus_results/NaCl-deformed-opt.extxyz
janus_results/NaCl-deformed-traj.extxyz


This allows us to visualise the optimisation:

In [9]:
from ase.io import read
from weas_widget import WeasWidget

traj = read("janus_results/NaCl-deformed-traj.extxyz", index=":")

v=WeasWidget()
v.from_ase(traj)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

WeasWidget(children=(BaseWidget(atoms=[{'species': {'Na': 'Na', 'Cl': 'Cl'}, 'cell': [5.63, 0.0, 0.0, 0.0, 5.6…

## Cell optimisation

We can also choose to modify the cell vectors during the optimisation. To allow only the cell lengths to change, we can run:

In [10]:
! janus geomopt --struct ../data/NaCl-deformed.xyz --write-traj --opt-cell-lengths --no-tracker

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

As before, we can visualise this trajectory:

In [11]:
from ase.io import read
from weas_widget import WeasWidget

traj = read("janus_results/NaCl-deformed-traj.extxyz", index=":")

v=WeasWidget()
v.from_ase(traj)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

WeasWidget(children=(BaseWidget(atoms=[{'species': {'Na': 'Na', 'Cl': 'Cl'}, 'cell': [5.63, 0.0, 0.0, 0.0, 5.6…

We can also allow the cell angles to change:

In [12]:
! janus geomopt --struct ../data/NaCl-deformed.xyz --write-traj --opt-cell-fully --no-tracker

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

Visualising this:

In [13]:
from ase.io import read
from weas_widget import WeasWidget

traj = read("janus_results/NaCl-deformed-traj.extxyz", index=":")

v=WeasWidget()
v.from_ase(traj)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

WeasWidget(children=(BaseWidget(atoms=[{'species': {'Na': 'Na', 'Cl': 'Cl'}, 'cell': [5.63, 0.0, 0.0, 0.0, 5.6…

## Setting the unit cell filter

Cell optimisation is carried out by appling an ASE `filter` to the structure. By default, this is the `FrechetCellFilter`, but you may wish to apply others, such as the `ExpCellFilter`. This is passed as a string, and must correspond to a class defined in ASE.

Key word arguments can also be passed to these filters. For example, we can maintain constant volume using the following configuration file:

In [14]:
%%writefile geomopt_config_1.yml

struct: ../data/NaCl-deformed.xyz
opt_cell_fully: True
filter_func: ExpCellFilter
minimize_kwargs:
  filter_kwargs:
    constant_volume: True
tracker: False

Writing geomopt_config_1.yml


<div class="alert alert-block alert-info">
<b>Tip:</b> This is equivalent to:

--struct ../data/NaCl-deformed.xyz --opt-cell-fully --filter-func ExpCellFilter --minimize-kwargs "{'filter_kwargs': {'constant_volume' : True}" --no-tracker
</div>


In [15]:
! janus geomopt --config geomopt_config_1.yml

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

Visualising this:

In [16]:
from ase.io import read
from weas_widget import WeasWidget

traj = read("janus_results/NaCl-deformed-traj.extxyz", index=":")

v=WeasWidget()
v.from_ase(traj)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

WeasWidget(children=(BaseWidget(atoms=[{'species': {'Na': 'Na', 'Cl': 'Cl'}, 'cell': [5.63, 0.0, 0.0, 0.0, 5.6…

## Constant pressure and symmetry refinement

We can also choose to optimise at a fixed pressure (in GPa), and refine the symmetry of the final structure:

In [17]:
! janus geomopt --struct ../data/NaCl-deformed.xyz --write-traj --pressure 10 --opt-cell-fully --symmetrize --no-tracker

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

Visualising this:

In [18]:
from ase.io import read
from weas_widget import WeasWidget

traj = read("janus_results/NaCl-deformed-traj.extxyz", index=":")

v=WeasWidget()
v.from_ase(traj)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

WeasWidget(children=(BaseWidget(atoms=[{'species': {'Na': 'Na', 'Cl': 'Cl'}, 'cell': [5.63, 0.0, 0.0, 0.0, 5.6…

## Comparing MACE to SevenNet

Finally, let's compare to the structure optimised by SevenNet:

In [19]:
%%writefile geomopt_config_mace.yml

struct: ../data/NaCl-deformed.xyz
arch: mace_mp
opt_cell_fully: True
minimize_kwargs:
  filter_kwargs:
    constant_volume: True
pressure: 10
file_prefix: janus_results/NaCl-mace
tracker: False

Writing geomopt_config_mace.yml


In [20]:
! janus geomopt --config geomopt_config_mace.yml
! janus geomopt --config geomopt_config_mace.yml --arch sevennet --file-prefix janus_results/NaCl-sevennet

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

Visualising the final structures:

In [21]:
from ase.io import read
from weas_widget import WeasWidget

mace_opt = read("janus_results/NaCl-mace-opt.extxyz")
sevennet_opt = read("janus_results/NaCl-sevennet-opt.extxyz")
opt_comparison = [mace_opt, sevennet_opt]

v=WeasWidget()
v.from_ase(opt_comparison)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

WeasWidget(children=(BaseWidget(atoms=[{'species': {'Na': 'Na', 'Cl': 'Cl'}, 'cell': [5.631828355006976, -0.10…